# Assignment Part 3: Segmenting and Clustering Neighborhoods in Toronto

## 1- Preparing Postal Codes data based on part 1 and 2

In [2]:
import pandas
Data = pandas.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') #import data from url
PostalCodesData = Data[0] #select first table as the postal codes data
PostalCodesData = PostalCodesData.loc[lambda x: x.Borough != 'Not assigned'] #Remove Boroughs are not assigned
PostalCodesData.reset_index(drop=True,inplace=True) #reset indexes in dataframe
PostalCodesData = PostalCodesData.groupby('Postal Code',as_index=False).agg(lambda x: ','.join(set(x.dropna()))) #Group postal codes based on Neighbourhood
GeoData=pandas.read_csv('http://cocl.us/Geospatial_data') #Import Latitude and Longitude from Geospatial file
PostalCodesData = pandas.merge(left=PostalCodesData,right=GeoData,on='Postal Code') #Merge postal codes and geographical coordinates data
PostalCodesData.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## 2- Select data from only boroughs that contain the word 'Toronto'

In [3]:
SelectedBoroughData = PostalCodesData[PostalCodesData['Borough'].str.contains('Toronto')].reset_index(drop=True)
SelectedBoroughData.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


## 3- Create map of Toronto and mark venues

In [6]:
#!conda install -c conda-forge geopy --yes
#!pip install folium
from geopy.geocoders import Nominatim
import folium

# Extract geograpical coordinate of Toronto
Address = 'Toronto'
Geo = Nominatim(user_agent="ny_explorer")
Location = Geo.geocode(Address)
Latitude = Location.latitude
Longitude = Location.longitude

# Create map of Toronoto and Boroughs' markers
TorontoMap = folium.Map(location=[Latitude, Longitude], zoom_start=12)
for x, y, b, n in zip(SelectedBoroughData['Latitude'], SelectedBoroughData['Longitude'],\
                      SelectedBoroughData['Borough'], SelectedBoroughData['Neighbourhood']):
    Label = '{}, {}'.format(n, b)
    Label = folium.Popup(Label, parse_html=True)
    folium.CircleMarker(
        [x, y],
        radius=5,
        popup=Label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMap);
TorontoMap

## 4- Define a function returns categories of each venues

### The function getNearbyVenues() gets geographical coordinates of whole venues and returns category of each venues by foursquare api.

In [31]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'Client ID', 
            'Client Secret', 
            '20200109', 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pandas.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Call getNearbyVenues() for venues in Toronto

In [36]:
#TorontoVenues = getNearbyVenues(names=SelectedBoroughData['Neighbourhood'],
 #                                  latitudes=SelectedBoroughData['Latitude'],
  #                                 longitudes=SelectedBoroughData['Longitude'])
TorontoVenues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
5,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop


### Find out how many unique categories can be curated from all the returned venues

In [37]:
TorontoVenuesData=TorontoVenues
TorontoVenues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(TorontoVenues['Venue Category'].unique())))

There are 235 uniques categories.


In [38]:
VenueCategories = pandas.get_dummies(TorontoVenues[['Venue Category']], prefix="", prefix_sep="") #One hot encoding
VenueCategories['Neighborhood'] = TorontoVenues['Neighborhood'] #Add neighborhood column back to dataframe
fixed_columns = [VenueCategories.columns[-1]] + list(VenueCategories.columns[:-1]) #Move neighborhood column to the first column
VenueCategories = VenueCategories[fixed_columns]
VenueCategories.head(10)

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
VenueCategoriesGrouped = VenueCategories.groupby('Neighborhood').mean().reset_index()
VenueCategoriesGrouped.head(5)

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.0


# 5- Create a new dataframe and display the top 10 venues for each neighborhood

In [45]:
import numpy
num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood'] #Create columns according to number of top venues
for ind in numpy.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
VenuesSorted = pandas.DataFrame(columns=columns) #Create a new dataframe
VenuesSorted['Neighborhood'] = VenueCategoriesGrouped['Neighborhood']

for ind in numpy.arange(VenueCategoriesGrouped.shape[0]):
    VenuesSorted.iloc[ind, 1:] = return_most_common_venues(VenueCategoriesGrouped.iloc[ind, :], num_top_venues)

VenuesSorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Restaurant,Seafood Restaurant,Cheese Shop,Bistro,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Gym,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Thai Restaurant,Miscellaneous Shop
5,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Restaurant,Baby Store,Candy Store,Nightclub,Coffee Shop
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Café,Pub,Men's Store
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Deli / Bodega
8,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Restaurant
9,Davisville North,Food & Drink Shop,Park,Pizza Place,Breakfast Spot,Hotel,Department Store,Sandwich Place,Wine Shop,Dim Sum Restaurant,Diner


# 6- Clustering venues to five neighbourhoods 

In [50]:
from sklearn.cluster import KMeans
kclusters = 5 #Set number of clusters
TorontoClustering = VenueCategoriesGrouped.drop('Neighborhood', 1)
TorontoKMeans = KMeans(n_clusters=kclusters, random_state=0).fit(TorontoClustering) #Run k-means clustering
TorontoKMeans.labels_[0:10] #Check cluster labels generated for each row in the dataframe

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 2], dtype=int32)

In [65]:
#VenuesSorted.insert(0, 'Cluster Labels', TorontoKMeans.labels_) #Add clustering labels
TorontoClustered = SelectedBoroughData
TorontoClustered = TorontoClustered.join(VenuesSorted.set_index('Neighborhood'), on='Neighbourhood') #Merge Toronto data with Clustering data to add latitude/longitude for each neighborhood
TorontoClustered

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Asian Restaurant,Pub,Health Food Store,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Cosmetics Shop,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4,Fast Food Restaurant,Gym,Pizza Place,Board Shop,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Dim Sum Restaurant,Swim School,Bus Line,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Food & Drink Shop,Park,Pizza Place,Breakfast Spot,Hotel,Department Store,Sandwich Place,Wine Shop,Dim Sum Restaurant,Diner
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,4,Coffee Shop,Sporting Goods Shop,Clothing Store,Gym / Fitness Center,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Diner,Mexican Restaurant,Cosmetics Shop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,4,Sandwich Place,Pizza Place,Dessert Shop,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Park,Trail,Tennis Court,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,4,Coffee Shop,Pub,Vietnamese Restaurant,Light Rail Station,Supermarket,Liquor Store,Sushi Restaurant,American Restaurant,Pizza Place,Bagel Shop


# 7- Visualize the resulting clusters

In [76]:
import matplotlib.cm as cm
import matplotlib.colors as colors
ClustersMap = folium.Map(location=[Latitude, Longitude], zoom_start=12)

# set color scheme for the clusters
x = numpy.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(numpy.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TorontoClustered['Latitude'], TorontoClustered['Longitude'], TorontoClustered['Neighbourhood'], TorontoClustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ClustersMap)
       
ClustersMap

# 8- Examine each cluster

## Cluster Zero

In [71]:
TorontoClustered.loc[TorontoClustered['Cluster Labels'] == 0, TorontoClustered.columns[[1] + list(range(5, TorontoClustered.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,0,Ice Cream Shop,Home Service,Garden,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


## Cluster One

In [72]:
TorontoClustered.loc[TorontoClustered['Cluster Labels'] == 1, TorontoClustered.columns[[1] + list(range(5, TorontoClustered.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Park,Trail,Tennis Court,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,Downtown Toronto,1,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


## Cluster Two

In [73]:
TorontoClustered.loc[TorontoClustered['Cluster Labels'] == 2, TorontoClustered.columns[[1] + list(range(5, TorontoClustered.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,2,Food & Drink Shop,Park,Pizza Place,Breakfast Spot,Hotel,Department Store,Sandwich Place,Wine Shop,Dim Sum Restaurant,Diner


## Cluster Three

In [74]:
TorontoClustered.loc[TorontoClustered['Cluster Labels'] == 3, TorontoClustered.columns[[1] + list(range(5, TorontoClustered.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Dim Sum Restaurant,Swim School,Bus Line,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
23,Central Toronto,3,Park,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Wine Shop,Dessert Shop,Electronics Store,Eastern European Restaurant,Donut Shop


## Cluster Four

In [75]:
TorontoClustered.loc[TorontoClustered['Cluster Labels'] == 4, TorontoClustered.columns[[1] + list(range(5, TorontoClustered.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Asian Restaurant,Pub,Health Food Store,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,East Toronto,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Cosmetics Shop,Brewery,Bubble Tea Shop
2,East Toronto,4,Fast Food Restaurant,Gym,Pizza Place,Board Shop,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub,Pet Store
3,East Toronto,4,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Cheese Shop,Clothing Store
6,Central Toronto,4,Coffee Shop,Sporting Goods Shop,Clothing Store,Gym / Fitness Center,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Diner,Mexican Restaurant,Cosmetics Shop
7,Central Toronto,4,Sandwich Place,Pizza Place,Dessert Shop,Gym,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Brewery,Restaurant
9,Central Toronto,4,Coffee Shop,Pub,Vietnamese Restaurant,Light Rail Station,Supermarket,Liquor Store,Sushi Restaurant,American Restaurant,Pizza Place,Bagel Shop
11,Downtown Toronto,4,Coffee Shop,Café,Bakery,Italian Restaurant,Pharmacy,Pizza Place,Restaurant,Pub,Yoga Studio,Beer Store
12,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Café,Pub,Men's Store
13,Downtown Toronto,4,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Theater,Wine Shop,Event Space,Performing Arts Venue
